<a href="https://colab.research.google.com/github/41371122h-lichi/lichi_thursday/blob/main/HW2_%E6%88%90%E7%B8%BE%E8%BC%B8%E5%85%A5%E5%8F%8A%E5%8F%8D%E9%A5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install google-generativeai

In [22]:
!pip install gradio

In [23]:
import pandas as pd
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import gspread
from google.oauth2.service_account import Credentials

In [24]:
api_key = userdata.get('gemini')

# 設定 API 金鑰
genai.configure(api_key=api_key)

# 選擇模型
model = genai.GenerativeModel('gemini-2.5-pro')

# 你的提問內容
prompt = "Explain how AI works in a few words"

# 發送請求並取得回應
response = model.generate_content(prompt)

# 印出回應的文字內容
print(response.text)

**By learning from data.**

Just like you learn from experience, AI learns by processing huge amounts of information, finding patterns, and using them to make decisions or predictions.


In [25]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1IcPEaEvIh0kGCGt5pklxNuQY89m3LLNt0qfyVNIj5LQ/edit?usp=sharing"
WORKSHEET_NAME = "工作表1"  # 你的工作表名稱
SERVICE_ACCOUNT_FILE = 'service_account.json'
REQUIRED_COLUMNS = ["學年度", "學期", "科目", "學期成績", "AI建議"]

# 從 Colab Secrets 中讀取 Gemini API 金鑰
# 確保你已經在 Colab Secrets 中設定了 'GEMINI_API_KEY'
try:
    genai.configure(api_key=userdata.get('gemini'))
    model = genai.GenerativeModel('gemini-2.5-pro')
except Exception as e:
    print(f"無法設定 Gemini API: {e}")
    model = None

In [26]:
try:
    genai.configure(api_key=userdata.get('gemini'))
    model = genai.GenerativeModel('gemini-2.5-pro')
    print("Gemini API 已成功設定。")
except Exception as e:
    print(f"無法設定 Gemini API，請檢查 Colab Secrets: {e}")
    model = None

# Google Sheets 服務帳號認證
# 請確認 service_account.json 檔案已上傳至 Colab
try:
    SERVICE_ACCOUNT_FILE = 'service_account.json'
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
    client = gspread.authorize(creds)
    print("Google Sheets 認證成功。")
    sheets_enabled = True
except Exception as e:
    print(f"Google Sheets 認證失敗，請檢查 service_account.json 檔案: {e}")
    sheets_enabled = False

# --- 2. 功能函式 ---
def get_ai_advice(course_name, grade, text_input):
    """根據單科成績和心得，產生 AI 建議"""
    if not model:
        return "AI 功能未啟用，請檢查 API 設定。"

    prompt = f"""
    你是一名專業的學術顧問。請根據以下學生的學期成績、個人心得，給予一份簡短且具體的學習建議，並指出亮點或潛在問題。

    科目：{course_name}
    學期成績：{grade}
    個人心得：{text_input}

    請以繁體中文回答，並使用條列式呈現，格式如下：
    1. 建議：[簡潔的建議]
    2. 總結：[總結亮點或潛在問題]
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"AI 建議產生失敗: {e}"

def get_overall_ai_advice(df, year, semester):
    """根據學期所有成績和心得，產生總體 AI 建議"""
    if not model:
        return "AI 功能未啟用，請檢查 API 設定。"

    # 篩選該學期所有資料
    df_semester = df[(df['學年度'] == year) & (df['學期'] == semester)].copy()
    if df_semester.empty:
        return f"學年度 {year} 的 {semester} 沒有找到任何成績記錄，無法給予總體建議。"

    all_grades = df_semester['學期成績'].str.cat(sep=", ")
    all_心得 = df_semester['個人心得'].str.cat(sep="\n---\n")

    prompt = f"""
    你是一名專業的學術顧問，請根據以下學期成績、所有科目的個人心得，給予一份總體性的學習建議和未來規劃。

    學年度：{year}
    學期：{semester}
    學期平均 GPA:{overall_gpa}

    所有科目的心得總結：
    {all_心得}

    請以繁體中文回答，並使用條列式呈現，格式如下：
    1. 整體表現總結：[簡潔的分析]
    2. 總體學習建議：[具體的學習建議]
    3. 未來規劃方向：[基於表現給予的未來方向]
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"AI 總體建議產生失敗: {e}"

def add_record_to_sheet(year, semester, subject, grade, text_input, is_final):
    """將單筆記錄寫入 Google Sheets，並根據選項處理 AI 建議和工作表切換"""
    if not sheets_enabled:
        return "❌ 錯誤：未連接 Google Sheets，請先上傳 service_account.json 檔案。"

    try:
        worksheet_name = f"{year}學年度{semester}"
        spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1IcPEaEvIh0kGCGt5pklxNuQY89m3LLNt0qfyVNIj5LQ/edit?usp=sharing")

        try:
            worksheet = spreadsheet.worksheet(worksheet_name)
        except gspread.exceptions.WorksheetNotFound:
            print(f"工作表 '{worksheet_name}' 不存在，正在創建...")
            worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows="100", cols="20")
            header = ["學年度", "學期", "科目", "學期成績", "個人心得", "AI建議"]
            worksheet.append_row(header)

        # **這裡直接使用 grade 字串，不再需要 gpa_to_number 轉換。**

        if is_final == "否":
            row_data = [year, semester, subject, grade, text_input, ""]
            worksheet.append_row(row_data)
            return f"""
            ✅ **登錄成功！**

            已新增一筆成績記錄。
            學年度: {year}
            學期: {semester}
            科目: {subject}
            學期成績: {grade}

            ---
            **分析中...**
            AI 總體建議將在您完成所有成績登錄，並選擇「是」後才會提供。
            """

        else:
            ai_advice = get_ai_advice(subject, grade, text_input)
            row_data = [year, semester, subject, grade, text_input, ai_advice]
            worksheet.append_row(row_data)

            data = worksheet.get_all_records()
            df = pd.DataFrame(data)
            overall_advice = get_overall_ai_advice(df, year, semester)

            return f"""
            ✅ **單科紀錄新增成功！**

            學年度: {year}
            學期: {semester}
            科目: {subject}
            學期成績: {grade}

            ---
            **單科AI建議:**
            {ai_advice}

            ---
            **學期總體AI建議:**
            {overall_advice}
            """
    except gspread.exceptions.SpreadsheetNotFound:
        return "❌ 錯誤：找不到指定的 Google Sheets，請檢查 URL。"
    except Exception as e:
        print(f"詳細錯誤訊息: {e}")
        return f"❌ 發生錯誤: {e}"

def handle_submit(year, semester, subject, grade, text_input, is_final):
    if not sheets_enabled:
        return "❌ 錯誤：未連接 Google Sheets，請先上傳 service_account.json 檔案。", "", "A+", "", "否"

    if is_final == "是":
        result = add_record_to_sheet(year, semester, subject, grade, text_input, is_final)
        return result, "", "A+", "", "否"
    else:
        result = add_record_to_sheet(year, semester, subject, grade, text_input, is_final)
        return result, "", "A+", "", "否"


# --- 3. Gradio 介面 ---
# 成績選項
grade_choices = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D", "E"]

# 學期選項
semester_choices = ["第一學期", "第二學期", "暑修"]

# 介面組件
with gr.Blocks() as demo:
    gr.Markdown("# 課程成績與 AI 建議紀錄工具")
    gr.Markdown("請輸入你的成績資訊，AI 將會提供個人化建議並將記錄儲存到 Google Sheets。")

    with gr.Row():
      year_input = gr.Textbox(label="學年度", placeholder="例如: 114(無須加單位)")
      semester_dropdown = gr.Dropdown(
        choices=semester_choices,
        label="學期",
        value="第一學期"
       )

    with gr.Row():
        subject_input = gr.Textbox(label="科目名稱", placeholder="例如: 程式語言")
        grade_dropdown = gr.Dropdown(
            choices=grade_choices,
            label="學期成績 (GPA)",
            value="A+"
        )

    text_input = gr.Textbox(
        lines=5,
        label="個人心得",
        placeholder="請輸入你的學習心得、遇到的困難或獲得的成就，寫得越詳細，AI給的建議就越精準唷！"
    )

    is_final_radio = gr.Radio(choices=["否","是"], value="否", label="是否已完成該學期所有成績登錄？")

    submit_button = gr.Button("新增紀錄並取得AI建議")

    # 使用 gr.State 來暫存學年度和學期
    year_state = gr.State()
    semester_state = gr.State()

    output_text = gr.Textbox(label="處理結果", interactive=False)

    submit_button.click(
        fn=handle_submit,
        inputs=[year_input, semester_dropdown, subject_input, grade_dropdown, text_input, is_final_radio],
        outputs=[output_text, subject_input, grade_dropdown, text_input, is_final_radio]
    )

# 啟動 Gradio 介面
demo.launch(share=True)

Gemini API 已成功設定。
Google Sheets 認證成功。
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3b0b4da9a53c00f39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
